## 克隆仓库并安装依赖

In [ ]:
# #@title Environment Setup

from pathlib import Path

OPTIONS = {}

USE_GOOGLE_DRIVE = True  #@param {type:"boolean"}
OPTIONS['USE_GOOGLE_DRIVE'] = USE_GOOGLE_DRIVE

current_dir = !pwd
WORKSPACE = f"{current_dir[0]}/lora-scripts"

if OPTIONS['USE_GOOGLE_DRIVE']:
    !echo "Mounting Google Drive..."
    %cd /

    from google.colab import drive
    drive.mount('/content/drive')

    WORKSPACE = "/content/drive/MyDrive/lora-scripts"
    %cd /content/drive/MyDrive

![ ! -d $WORKSPACE ] && echo -= Initial setup lora-scripts =- && git clone --recurse-submodules https://github.com/Akegarasu/lora-scripts.git
%cd $WORKSPACE


!echo -= Install dependencies =-
!bash ./install.bash

# !pip3 install accelerate
# !pip3 install einops transformers>=4.28.1 safetensors>=0.4.2 aiohttp pyyaml Pillow scipy tqdm psutil tokenizers>=0.13.3
# !pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
# !pip3 install torchsde
# !pip3 install kornia>=0.7.1 spandrel soundfile sentencepiece



Mounting Google Drive...
/
Mounted at /content/drive
/content/drive/MyDrive
-= Initial setup lora-scripts =-
Cloning into 'lora-scripts'...
remote: Enumerating objects: 2429, done.
remote: Counting objects: 100% (125/125), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 2429 (delta 107), reused 86 (delta 86), pack-reused 2304 (from 2)
Receiving objects: 100% (2429/2429), 1.87 MiB | 9.41 MiB/s, done.
Resolving deltas: 100% (1509/1509), done.
Updating files: 100% (275/275), done.
Submodule 'frontend' (https://github.com/hanamizuki-ai/lora-gui-dist) registered for path 'frontend'
Submodule 'mikazuki/dataset-tag-editor' (https://github.com/Akegarasu/dataset-tag-editor) registered for path 'mikazuki/dataset-tag-editor'
Cloning into '/content/drive/MyDrive/lora-scripts/frontend'...
remote: Enumerating objects: 2405, done.        
remote: Counting objects: 100% (1529/1529), done.        
remote: Compressing objects: 100% (829/829), done.        
remote: Total 2405 (delta 

## 下载模型

In [ ]:
!wget -c https://huggingface.co/Kijai/flux-fp8/resolve/main/flux1-dev-fp8.safetensors -P /content/models/unet/

In [ ]:
!wget -c https://huggingface.co/comfyanonymous/flux_text_encoders/resolve/main/clip_l.safetensors -P /content/models/text_encoders/

In [ ]:
!wget -c https://huggingface.co/comfyanonymous/flux_text_encoders/resolve/main/t5xxl_fp8_e4m3fn.safetensors -P /content/models/text_encoders/

In [ ]:
!wget -c https://huggingface.co/black-forest-labs/FLUX.1-dev/resolve/main/ae.safetensors -P /content/models/vae/

### Run ComfyUI with cloudflared (Recommended Way)




In [ ]:
!wget -P ~ https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i ~/cloudflared-linux-amd64.deb

import subprocess
import threading
import time
import socket
import urllib.request

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI finished loading, trying to launch cloudflared (if it gets stuck here cloudflared is having issues)\n")

  p = subprocess.Popen(["cloudflared", "tunnel", "--url", "http://127.0.0.1:{}".format(port)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  for line in p.stderr:
    l = line.decode()
    if "trycloudflare.com " in l:
      print("This is the URL to access ComfyUI:", l[l.find("http"):], end='')
    #print(l, end='')


threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

!python main.py --dont-print-server

### Run ComfyUI with localtunnel




In [ ]:
!npm install -g localtunnel

import subprocess
import threading
import time
import socket
import urllib.request

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI finished loading, trying to launch localtunnel (if it gets stuck here localtunnel is having issues)\n")

  print("The password/enpoint ip for localtunnel is:", urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))
  p = subprocess.Popen(["lt", "--port", "{}".format(port)], stdout=subprocess.PIPE)
  for line in p.stdout:
    print(line.decode(), end='')


threading.Thread(target=iframe_thread, daemon=True, args=(28000,)).start()

!bash run_gui.sh --dont-print-server

### Run ComfyUI with colab iframe (use only in case the previous way with localtunnel doesn't work)

You should see the ui appear in an iframe. If you get a 403 error, it's your firefox settings or an extension that's messing things up.

If you want to open it in another window use the link.

Note that some UI features like live image previews won't work because the colab iframe blocks websockets.

In [ ]:
import threading
import time
import socket
def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  from google.colab import output
  output.serve_kernel_port_as_iframe(port, height=1024)
  print("to open it in a window you can open this link here:")
  output.serve_kernel_port_as_window(port)

threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

!python main.py --dont-print-server